### #1. Using the dataset of article open-access prices paid by the WELLCOME Trust between 2012 and 2013, determine the five most common journals and the total articles for each. 

In [247]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re
import matplotlib
%matplotlib inline
from scipy.stats import mstats
import locale

# source file encoding is western mac os roman, according an app I opened it with.
# default encoding is utf-8

locale.getpreferredencoding()

'UTF-8'

In [248]:
#because source file encoding is Western Mac os Roman, file loads when set to latin-1.
df = pd.read_csv('/thinkful/data_files/wellcome/wellcome.csv', encoding='latin-1')

In [249]:
df.count()

PMID/PMCID                                             1928
Publisher                                              2127
Journal title                                          2126
Article title                                          2127
COST (£) charged to Wellcome (inc VAT when charged)    2127
dtype: int64

In [250]:
# Drop the PMID/PMCID column since it just gets in the way and will make it easier to drop all
# NaN values later.
df.drop('PMID/PMCID', axis=1, inplace=True)

In [251]:
# If I type "df", I want to see all the rows.
pd.options.display.max_rows = 2127

In [252]:
# Change column names to reduce error chances  
df.rename(columns={df.columns[3]: 'Cost_to_Wellcome_in_GBP_plus_VAT'}, inplace=True)
df.rename(columns={df.columns[2]: 'Article_Title'}, inplace=True)
df.rename(columns={df.columns[1]: 'Journal_Title'}, inplace=True)
df.rename(columns={df.columns[0]: 'Publisher'}, inplace=True)
df.head()

,Publisher,Journal_Title,Article_Title,Cost_to_Wellcome_in_GBP_plus_VAT
0,CUP,Psychological Medicine,Reduced parahippocampal cortical thickness in ...,£0.00
1,ACS,Biomacromolecules,Structural characterization of a Model Gram-ne...,£2381.04
2,ACS,J Med Chem,"Fumaroylamino-4,5-epoxymorphinans and related ...",£642.56
3,ACS,J Med Chem,Orvinols with mixed kappa/mu opioid receptor a...,£669.64
4,ACS,J Org Chem,Regioselective opening of myo-inositol orthoes...,£685.88


In [253]:
# Remove leading and trailing whitespace

df['Publisher'] = df['Publisher'].str.strip()
df['Journal_Title'] = df['Journal_Title'].str.strip()
df['Article_Title'] = df['Article_Title'].str.strip()
df['Cost_to_Wellcome_in_GBP_plus_VAT'] = df['Cost_to_Wellcome_in_GBP_plus_VAT'].str.strip()

df.head(10)

,Publisher,Journal_Title,Article_Title,Cost_to_Wellcome_in_GBP_plus_VAT
0,CUP,Psychological Medicine,Reduced parahippocampal cortical thickness in ...,£0.00
1,ACS,Biomacromolecules,Structural characterization of a Model Gram-ne...,£2381.04
2,ACS,J Med Chem,"Fumaroylamino-4,5-epoxymorphinans and related ...",£642.56
3,ACS,J Med Chem,Orvinols with mixed kappa/mu opioid receptor a...,£669.64
4,ACS,J Org Chem,Regioselective opening of myo-inositol orthoes...,£685.88
5,ACS,Journal of Medicinal Chemistry,Comparative Structural and Functional Studies ...,£2392.20
6,ACS,Journal of Proteome Research,Mapping Proteolytic Processing in the Secretom...,£2367.95
7,ACS,Mol Pharm,Quantitative silencing of EGFP reporter gene b...,£649.33
8,ACS (Amercian Chemical Society) Publications,ACS Chemical Biology,A Novel Allosteric Inhibitor of the Uridine Di...,£1294.59
9,ACS (Amercian Chemical Society) Publications,ACS Chemical Biology,Chemical proteomic analysis reveals the drugab...,£1294.78


In [254]:
# Replace variations of Publisher and Journal Titles with unique standardized spellings.

# FIX 'Publisher' COLUMN
df['Publisher'].replace({
        'CUP' : 'Cambridge University Press',
        'Cambridge Uni Press' : 'Cambridge University Press',
        'Cambridge Univ Press' : 'Cambridge University Press',
        'Cambridge Journals' : 'Cambridge University Press',
        'ACS (Amercian Chemical Society) Publications' : 'ACS',
        'ACS Publications' : 'ACS',
        'American Chemical Society' : 'ACS',
        'AMERICAN CHEMICAL SOCIETY' : 'ACS',
        'American Chemical Society Publications' : 'ACS',
        'The American Physiological Society' : 'American Physiological Society',
        'American Psychiatric Publishing' : 'American Psychiatric Association',
        'The American Society for Biochemistry and Molecular Biology' : 'American Society for Biochemistry and Molecular Biology',
        'The American Society for Biochemistry and Molecular Biology, Inc' : 'American Society for Biochemistry and Molecular Biology',
        'American Soc for Biochemistry and Molecular Biology' : 'American Society for Biochemistry and Molecular Biology',
        'American Society for Biochemistry and Molecular Biolgy' : 'American Society for Biochemistry and Molecular Biology',
        'AMBSB (The American Society for Biochemistry and Molecular Biology)' : 'American Society for Biochemistry and Molecular Biology',
        'ASBMB' : 'American Society for Biochemistry and Molecular Biology',
        'ASM' : 'American Society for Microbiology',
        'American Society of Microbiology' : 'American Society for Microbiology',
        'ASM (American Society for Microbiology)' : 'American Society for Microbiology',
        'Cenveo Publisher Servies/ASM JV1' : 'American Society for Microbiology',
        'American Society of Haematology' : 'American Society of Hematology',
        'American Society of Hamatology' : 'American Society of Hematology',
        'American Society of Human Genetics (Elsevier)' : 'Elsevier',
        'ELSEVIER' : 'Elsevier',
        'Cell Press' : 'Elsevier',
        'Elseveier Science' : 'Elsevier',
        'Elsevier (Cell Press)' : 'Elsevier',
        'Elsevier / Cell Science' : 'Elsevier',
        'Elsevier B.V.' : 'Elsevier',
        'Elsevier Ltd' : 'Elsevier',
        'Elsevier/Cell Press' : 'Elsevier',
        'The Boulevard' : 'Elsevier',
        'Cenveo Publisher services' : 'Cenveo',
        'Cadmus Journal Services' : 'Cenveo',
        'Cadmus' : 'Cenveo',
        'CADMUS JOURNAL SERVICE' : 'Cenveo',
        'CADMUS JOURNAL SERVICES' : 'Cenveo',
        'Camdus Journal Services' : 'Cenveo',
        'ASBMB/Cadmus' : 'Cenveo',
        'ASBMB Cadmus' : 'Cenveo',
        'ASBMB/Cenveo Publisher Services' : 'Cenveo',
        'ASBMC /CENVEO' : 'Cenveo',
        'Benthan Science Publishers' : 'Bentham Science Publishers',
        'Biochem Journal' : 'Portland Press',
        'Portland press' : 'Portland Press',
        'PORTLAND PRESS LTD' : 'Portland Press',
        'Portland Press Ltd' : 'Portland Press',
        'PORTLAND PRESS LTD' : 'Portland Press',
        'Springer' : 'Springer Nature',
        'BMC' : 'Springer Nature',
        'BioMed Central' : 'Springer Nature',
        'BioMed central' : 'Springer Nature',
        'Biomed Central' : 'Springer Nature',
        'BioMed Central Limited' : 'Springer Nature',
        'BioMed Central Ltd' : 'Springer Nature',
        'Springer - Verlag GMBH' : 'Springer Nature',
        'Springer Science + Business Media' : 'Springer Nature',
        'Springer Verlag' : 'Springer Nature',
        'Springer-Verlag GmbH' : 'Springer Nature',
        'SPRINGER-VERLAG GMBH' : 'Springer Nature',
        'Humana Press (Springer Imprint)' : 'Springer Nature',
        'Springer-Verlag GMBH & Ci' : 'Springer Nature',
        'Springer-Verlag GmbH, Heidelberger Platz 3, D-14197 Berlin' : 'Springer Nature',
        'Nature' : 'Springer Nature',
        'Nature PG' : 'Springer Nature',
        'Nature Publishing' : 'Springer Nature',
        'Nature Publishing Group' : 'Springer Nature',
        'NATURE PUBLISHING GROUP LTD' : 'Springer Nature',
        'NPG' : 'Springer Nature',
        'Palgrave MacMillan' : 'Springer Nature',
        'Informa Healthcare' : 'Informa',
        'Informa Healthcare communications' : 'Informa',
        'Taylor & Francis' : 'Informa',
        'Taylor & Francis Journals' : 'Informa',
        'Taylor and Francis' : 'Informa',
        'Copyright Clearace Center' : 'Informa',
        'T&F' : 'Informa',
        'Byophysical Society' : 'Biophysical Society',
        'BioScientifica' : 'Bioscientifica',
        'Blackwell Publishing Ltd/' : 'Wiley',
        'John Wiley' : 'Wiley',
        'JOHN WILEY & SONS' : 'Wiley',
        'John Wiley & Sons' : 'Wiley',
        'John Wiley & Sons Inc' : 'Wiley',
        'John Wiley & Sons Ltd' : 'Wiley',
        'John Wiley & Sons, Inc.' : 'Wiley',
        'John Wiley and Sons' : 'Wiley',
        'John Wiley and Sons Ltd' : 'Wiley',
        'Open Access Reg Ltd' : 'Wiley',
        'Wiley & Son' : 'Wiley',
        'Wiley Blackwell' : 'Wiley',
        'Wiley Online Library' : 'Wiley',
        'Wiley Subscription Services' : 'Wiley',
        'Wiley Subscription Services Inc' : 'Wiley',
        'Wiley Subscription Services Inc.' : 'Wiley',
        'Wiley Subscription Serviices Inc' : 'Wiley',
        'Wiley VCH' : 'Wiley',
        'Wiley/Blackwell' : 'Wiley',
        'Wiley-Blackwell' : 'Wiley',
        'Wiley-Blackwell, John Wiley & Sons' : 'Wiley',
        'Wiley-VCH' : 'Wiley',
        'Wliey-Blackwell' : 'Wiley',
        'BMJ Group' : 'BMJ',
        'BMJ group' : 'BMJ',
        'BMJ Journals' : 'BMJ',
        'BMJ Publishing Group' : 'BMJ',
        'BMJ PUBLISHING GROUP' : 'BMJ',
        'BMJ Publishing Group Ltd' : 'BMJ',
        'BMJ Publishing Group Ltd & British Thoracic Society' : 'BMJ',
        'British Medical Journal' : 'BMJ',
        'COACTION' : 'Co-Action Publishing',
        'Cold Spring Habour Press' : 'Cold Spring Harbor Laboratory Press',
        'Cold Spring Harbor' : 'Cold Spring Harbor Laboratory Press',
        'Cold Spring Harbor Press' : 'Cold Spring Harbor Laboratory Press',
        'Cold Spring Harbor Publications' : 'Cold Spring Harbor Laboratory Press',
        'CSHLP' : 'Cold Spring Harbor Laboratory Press',
        'Company of Biologist' : 'The Company of Biologists',
        'Company of Biologists' : 'The Company of Biologists',
        'Company of Biologists Ltd' : 'The Company of Biologists',
        'The company of Biolgists' : 'The Company of Biologists',
        'THE COMPANY OF BIOLOGISTS' : 'The Company of Biologists',
        'The company of Biologists' : 'The Company of Biologists',
        'The Company of Biologists Ltd' : 'The Company of Biologists',
        'Darmouth Journal Services' : 'The Sheridan Group',
        'Dartmouth Journal Services' : 'The Sheridan Group',
        'Dartmouth Journals' : 'The Sheridan Group',
        'Sheridan Press' : 'The Sheridan Group',
        'The Sheridan Press' : 'The Sheridan Group',
        'ABC' : 'Endocrine Society',
        'The Endocrine Society' : 'Endocrine Society',
        'THE ENDOCRINE SOCIETY' : 'Endocrine Society',
        'The Endrocrine Society' : 'Endocrine Society',
        'European Society of Endocrinolog' : 'European Society of Endocrinology',
        'FASEB' : 'Federation of American Societies for Experimental Biology',
        'Federation of American Societies for Experimental Biology (FASEB)' : 'Federation of American Societies for Experimental Biology',
        'Federation of the American Society of Experimental Biology' : 'Federation of American Societies for Experimental Biology',
        'Frontiers' : 'Frontiers Media',
        'Frontiers Media SA' : 'Frontiers Media',
        'Frontiers Research Foundation' : 'Frontiers Media',
        'Future Medicine Ltd' : 'Future Medicine',
        'Hindawi Publishing Corporation' : 'Hindawi',
        'Impact Journals LLC' : 'Impact Journals',
        'Institute of Physics' : 'IOP Publishing',
        'International Union Against tuberculosis and Lung Disease' : 'International Union Against Tuberculosis and Lung Disease',
        'International Union of Crystallography (iucr)' : 'International Union of Crystallography',
        'Journal of the American Physiological Proceedings of National Academy of Sciences' : 'NAS',
        'National Academy of Sciences' : 'NAS',
        'National Academy of Sciences of the United States of America' : 'NAS',
        'National Academy of Sciences USA' : 'NAS',
        'National Academy of Sciences, USA' : 'NAS',
        'Proceedings of the National Academy of Sciences (PNAS)' : 'NAS',
        'JoVE' : 'Jove',
        'Journal of Visualized Experiments' : 'Jove',
        'The Journal of Visualized Experiments' : 'Jove',
        'Journal of Visualized Experiements' : 'Jove',
        'Journal of Visulaized expermiments' : 'Jove',
        'MY JOVE CORP' : 'Jove',
        'My JOVE corporation' : 'Jove',
        'MYJoVE Corporation' : 'Jove',
        'KARGER' : 'Karger',
        'Landes Biosciences' : 'Landes Bioscience',
        'MARY ANN LIEBERT INC' : 'Mary Ann Liebert',
        'Mary Ann Liebert, Inc. Publishers' : 'Mary Ann Liebert',
        'MIT PRESS OPEN ACCESS' : 'MIT Press',
        'OUP' : 'Oxford University Press',
        'Oxford Journals' : 'Oxford University Press',
        'Oxford Journals (OUP)' : 'Oxford University Press',
        'Oxford Univ Press' : 'Oxford University Press',
        'OXFORD UNIVERSITY PRESS' : 'Oxford University Press',
        'Oxford University Press (OUP)' : 'Oxford University Press',
        'Oxford Univesity Press' : 'Oxford University Press',
        'Plos' : 'PLOS',
        'PLoS' : 'PLOS',
        'PLoS (Public Library of Science)' : 'PLOS',
        'Public Library of Science' : 'PLOS',
        'public.service.co.uk' : 'PSCA International',
        'PubMed Central' : 'PubMed',
        'The Royal College of Psychiatrists' : 'Royal College of Psychiatrists',
        'The Royal Society' : 'Royal Society',
        'The royal Society' : 'Royal Society',
        'Royal Society for Chemistry' : 'Royal Society of Chemistry',
        'RSC' : 'Royal Society of Chemistry',
        'RSC Publishing' : 'Royal Society of Chemistry',
        'Sage' : 'SAGE',
        'Sage Publications' : 'SAGE',
        'SAGE Publications' : 'SAGE',
        'Sage Publications Inc' : 'SAGE',
        'Sage Publications Ltd' : 'SAGE',
        'Sage Publishers' : 'SAGE',
        'Sage Publishing' : 'SAGE',
        'Society for General Microbiology' : 'Microbiology Society',
        'Society for Genermal Microbiology' : 'Microbiology Society',
        'Society of General Microbiology' : 'Microbiology Society',
        'Society of Leukocyte Biology' : 'Society for Leukocyte Biology',
        'Society for Neurosciences' : 'Society for Neuroscience',
        'SOCIETY OF NEURO SCIENCES' : 'Society for Neuroscience',
        'Society of Neuroscience' : 'Society for Neuroscience',
        'The American Chemical Society Petroleum Research Trust' : 'ACS',
        'The American Society of Pediatrics' : 'American Academy of Pediatrics',
        'Wolters Kluwer Health' : 'Wolters Kluwer',
        'Wolters Kluwer N.V./Lippinott' : 'Wolters Kluwer',
        'Wolters Kluwers' : 'Wolters Kluwer',
        'LWW' : 'Wolters Kluwer'
       
   
 }, inplace=True)


# FIX 'Journal_Title' COLUMN
df['Journal_Title'].replace({
    
        'Pyschological Medicine' : 'Psychological Medicine',
        'J Med Chem' : 'Journal of Medicinal Chemistry',
        'J Med Chem.' : 'Journal of Medicinal Chemistry',
        'J Biol Chem.' : 'Journal of Biological Chemistry',
        'Journal of Biol Chem' : 'Journal of Biological Chemistry',
        'J Biol Chemistry' : 'Journal of Biological Chemistry',
        'Journal of Biol Chem' : 'Journal of Biological Chemistry',
        'Jnl Biological Chemistry' : 'Journal of Biological Chemistry',
        'Journal of biological chemistry' : 'Journal of Biological Chemistry',
        'JOURNAL OF BIOLOGICAL CHEMISTRY' : 'Journal of Biological Chemistry',
        'The Journal of Biological Chemistry' : 'Journal of Biological Chemistry',
        'The journal of Biological Chemistry' : 'Journal of Biological Chemistry',
        'BIOLOGICAL CHEMISTRY' : 'Journal of Biological Chemistry',
        'Biologicial Chemistry' : 'Journal of Biological Chemistry',
        'Journal Biological Chemistry' : 'Journal of Biological Chemistry',
        'Journal of Immunology' : 'The Journal of Immunology',
        'J Immunol' : 'The Journal of Immunology',
        'ACS NANO' : 'ACS Nano',
        'American Chemical Society' : 'Journal of the American Chemical Society',
        'The Journal of Neurophysiology' : 'Journal of Neurophysiology',
        'American Journal of Psychiatry' : 'The American Journal of Psychiatry',
        'Americal Journal of Psychiatry' : 'The American Journal of Psychiatry',
        'Journal of Experimental Psychology: Animal Behaviour Process' : 'Journal of Experimental Psychology: Animal Behavior Processes',
        'Antimicrobial Agfents and Chemotherapy' : 'Antimicrobial Agents and Chemotherapy',
        'Antimicobial Agents and Chemotherapy' : 'Antimicrobial Agents and Chemotherapy',
        'JOURNAL OF BACTERIOLOGY' : 'Journal of Bacteriology',
        'Journal of Virol' : 'Journal of Virology',
        'Mbio' : 'mBio',
        'J Clin Microbiol' : 'Journal of Clinical Microbiology',
        'Jounral of Clinical Microbiology' : 'Journal of Clinical Microbiology',
        'Blood Journal 2012' : 'Blood',
        'THETHE AMERICAN JOURNAL OF HUMAN GENETICS' : 'The American Journal of Human Genetics',
        'American Journal of Human Genetics' : 'The American Journal of Human Genetics',
        'The Americal Journal of Human Genetics' : 'The American Journal of Human Genetics',
        'MCP (Molecular & Cellular Proteomics)' : 'Molecular and Cellular Proteomics',
        'Biochem Journal' : 'Biochemical Journal',
        'Biochemical Journals' : 'Biochemical Journal',
        'The Biochemical Journal' : 'Biochemical Journal',
        'BioMed Central' : 'Journal of Neuroinflammation',
        'Genome Biology' : 'BMC Genome Biology',
        'BMS Genomics' : 'BMC Genomics',
        'BMC Genomics.' : 'BMC Genomics',
        'BMC infectious diseases' : 'BMC Infectious Diseases',
        'International Journal of Behavioural Nutrition and Physical Activity' : 'International Journal of Behavioral Nutrition and Physical Activity',
        'Parasit Vectors.' : 'Parasites and Vectors',
        'RESPIRATORY RESEARCH' : 'Respiratory Research',
        'Hum Resour Health' : 'Human Resources for Health',
        'Implement Sci.' : 'Implementation Science',
        'Journal of molecular Endocrinology' : 'Journal of Molecular Endocrinology',
        'Reproduction(Cambridge, England)' : 'Reproduction',
        'BMJ OPEN' : 'BMJ Open',
        'BMJ open' : 'BMJ Open',
        'British Medical Journal' : 'BMJ',
        'British Journal of Opthalmology' : 'British Journal of Ophthalmology',
        'BJOPHTHALMOL' : 'British Journal of Ophthalmology',
        'Journal of Neurology, Neurosurgery and Psychiatry' : 'Journal of Neurology, Neurosurgery & Psychiatry',
        'Sex Transm Infect.' : 'Sexually Transmitted Infections',
        'Journal of Epidemiology and Community Health' : 'Journal of Epidemiology & Community Health',
        'THORAX an international journal for Respiratory Medicine' : 'Thorax',
        'The Journal of Neuroscience' : 'Journal of Neuroscience',
        'Parsitology' : 'Parasitology',
        'British Journal of the History of Science' : 'British Journal for the History of Science',  
        'Journal of Clinical Endocrinology' : 'The Journal of Clinical Endocrinology and Metabolism',
        'Journal of Clinical Endocrinology and Metabolism' : 'The Journal of Clinical Endocrinology and Metabolism',
        'Journal od Clinical Endocrinology' : 'The Journal of Clinical Endocrinology and Metabolism',
        'J Cell Sci.' : 'Journal of Cell Science',
        'BIOL OPEN' : 'Biology Open',
        'DEVELOPMENT CELL' : 'Developmental Cell',
        'Cell Press - Cell Reports' : 'Cell Reports',
        'American Journal of Clinical Nutrition' : 'The American Journal of Clinical Nutrition',
        'American Journal for Clinical Nutrition' : 'The American Journal of Clinical Nutrition',
        'GENETICS' : 'Genetics',
        'Journal of Nutrition' : 'The Journal of Nutrition',
        'PNAS' : 'Proceedings of the National Academy of Sciences of the United States of America',
        'PNAS USA' : 'Proceedings of the National Academy of Sciences of the United States of America',
        'Proceedings of the National Academy of Sciences (PNAS)' : 'Proceedings of the National Academy of Sciences of the United States of America',
        'Proceedings of the National Academy of Sciences of the USA' : 'Proceedings of the National Academy of Sciences of the United States of America',
        'PNAS (Proceedings of the National Academy of Sciences of the United States of America' : 'Proceedings of the National Academy of Sciences of the United States of America',
        'Proceddings of the National Academy of Sciences of USA' : 'Proceedings of the National Academy of Sciences of the United States of America',
        'Proceedings of the national Academy of Sciences' : 'Proceedings of the National Academy of Sciences of the United States of America',
        'Proceedings of the National Academy of Sciences' : 'Proceedings of the National Academy of Sciences of the United States of America',
        'Proc Natl Acad Sci U S A.' : 'Proceedings of the National Academy of Sciences of the United States of America',
        'National Academy of Sciences' : 'Proceedings of the National Academy of Sciences of the United States of America',
        'PNAS Author Publication' : 'Proceedings of the National Academy of Sciences of the United States of America',
        'Neuroimage' : 'NeuroImage',
        'Neurolmage' : 'NeuroImage',
        'THETHE AMERICAN JOURNAL OF HUMAN GENETICS' : 'The American Journal of Human Genetics',
        'American Journal of Human Genetics' : 'The American Journal of Human Genetics',
        'Biochimica et Bioohysica Acta - Molecular Basis of Disease' : 'BBA - Molecular Basis of Disease',
        'Biochimica et Biophysica Acta - Molecular Basis of Disease' : 'BBA - Molecular Basis of Disease',
        'Behavior Research and Therapy' : 'Behaviour Research and Therapy',
        'Biochimica et Bioohysica Acta - Gene Regulatory Mechanisms' : 'BBA - Gene Regulatory Mechanisms',
        'Biochimica et Biophysica Acta (BBA) - Molecular Cell Research' : 'BBA - Molecular Cell Research',
        'BIOLOGICAL PSYCHIATRY' : 'Biological Psychiatry',
        'Cell Journal' : 'Cell',
        'Cell Host & Microbe' : 'Cell Host and Microbe',
        'Chemsitry & Biology' : 'Chemistry & Biology',
        'Consciousness & Cognition' : 'Consciousness and Cognition',
        'CURRENT BIOLOGY' : 'Current Biology',
        'Current Opinions in Neurobiology' : 'Current Opinion in Neurobiology',
        'European Journal of cell Biology' : 'European Journal of Cell Biology',
        'Febs Letters' : 'FEBS Letters',
        'GENE' : 'Gene',
        'Health and Place' : 'Health & Place',
        'Immnunobiology' : 'Immunobiology',
        'Int Journal for Parasitology' : 'International Journal for Parasitology',
        'International Journal of Parasitology' : 'International Journal for Parasitology',
        'International journal of applied earth observation and geoinformation' : 'International Journal of Applied Earth Observation and Geoinformation',
        'J Mol Biol' : 'Journal of Molecular Biology',
        'Mol Biol' : 'Journal of Molecular Biology',
        'MOLECULAR BIOLOGY' : 'Journal of Molecular Biology',
        'J Steroid Biochem Mol Biol' : 'The Journal of Steroid Biochemistry & Molecular Biology',
        'JOURNAL OF ALLERGY & CLINICAL IMMUNOLOGY' : 'The Journal of Allergy and Clinical Immunology',
        'Journal of Allergy and Clinical Immunology' : 'The Journal of Allergy and Clinical Immunology',
        'Journal of allergy and clinical immunology' : 'The Journal of Allergy and Clinical Immunology',
        'JOURNAL OF ALLERGY & CLINICAL IMMUNOLOGY' : 'The Journal of Allergy and Clinical Immunology',
        'Journal of Behaviour therapy and Experimental Psychiatry' : 'Journal of Behavior Therapy and Experimental Psychiatry',
        'Journal of cycstic fibrosis' : 'Journal of Cystic Fibrosis',
        'Mechanisms of Ageing and Development' : 'Journal of Mechanisms of Ageing and Development',
        'Journal of paediatric Urology' : 'Journal of Pediatric Urology',
        'Lancet' : 'The Lancet',
        'Lancet Global Health' : 'The Lancet Global Health',
        'Lancet Infect Dis' : 'The Lancet Infectious Diseases',
        'Microbes Infect' : 'Microbes and Infection',
        'Molecluar & Cellular Endocrinology' : 'Molecular and Cellular Endocrinology',
        'Molecular & Biochemical Parasitology' : 'Molecular and Biochemical Parasitology',
        'N Biotechnol.' : 'New Biotechnology',
        'NeuroImage: Clinical' : 'Neuroimage: Clinical',
        'Neuroimage: clinical' : 'Neuroimage: Clinical',
        'Neurophysiologia' : 'Neuropsychologia',
        'Parkinsonism and Related Disorders' : 'Parkinsonism & Related Disorders',
        'Protein Expr Purif.' : 'Protein Expression and Purification',
        'RESEARCH IN VETERINARY SCIENCE' : 'Research in Veterinary Science',
        'Social Science and Medicine' : 'Social Science & Medicine',
        'Studies in History and Philosophy of Science Part C' : 'Studies in History and Philosophy of Science Part C: Studies in History and Philosophy of Biological and Biomedical Sciences',
        'The Vet. Journal' : 'The Veterinary Journal',
        'Trends in Cognitive Science' : 'Trends in Cognitive Sciences',
        'Trends in Neuroscience' : 'Trends in Neurosciences',
        'Math Biosci' : 'Mathematical Biosciences',
        'Curr Biol.' : 'Current Biology',
        'FASEB Journal' : 'The FASEB Journal',
        'Haematologica/The Haematology Journal' : 'Haematologica',
        'JOURNAL OF ALLERGY' : 'Journal of Allergy',
        'OXIDATIVE MEDICINE & CELLULAR LONGEVITY' : 'Oxidative Medicine and Cellular Longevity',
        'Expert Reviews in Anti-infective Chemotherapy' : 'Expert Review of Anti-infective Therapy',
        'IJTLD' : 'The International Journal of Tuberculosis and Lung Disease',
        'International Journal of Tuberculosis and Lung Disease' : 'The International Journal of Tuberculosis and Lung Disease',
        'Acta Crystallographica Section D: Biological Crystallography' : 'Acta Crystallographica Section D',
        'Acta Crystallographica, Section D' : 'Acta Crystallographica Section D',
        'Acta Crystallography D' : 'Acta Crystallographica Section D',
        'Acta Crystallographica Section D,  Biological Crystallography' : 'Acta Crystallographica Section D',
        'Acta D' : 'Acta Crystallographica Section D',
        'Acta F' : 'Acta Crystallographica Section F',
        'Acta Crystallographica Section F: Structural Biology and Crystallization Communications' : 'Acta Crystallographica Section F',
        'J Synchrotron Radiat.' : 'Journal of Synchrotron Radiation',
        'Journal of Alzheimer Disease' : 'Journal of Alzheimer\'s Disease',
        'Journal of Pathology' : 'The Journal of Pathology',
        'Eur J Immunol' : 'European Journal of Immunology',
        'Journal of Depression & Anxiety' : 'Depression & Anxiety',
        'Depression and Anxiety' : 'Depression & Anxiety',
        'Clinical and Experimental Allergy' : 'Clinical & Experimental Allergy',
        'Trop Med Int Health' : 'Tropical Medicine & International Health',
        'Tropical Medicine and International Health' : 'Tropical Medicine & International Health',
        'Journal of Physiology' : 'The Journal of Physiology',
        'Epigentics' : 'Epigenetics',
        'Antioxidants and Redox Signaling' : 'Antioxidants & Redox Signaling',
        'Tissue Engineering: part A' : 'Tissue Engineering Part A',
        'International journal of molecular sciences' : 'International Journal of Molecular Sciences',
        'JERPH' : 'International Journal of Environmental Research and Public Health',
        'The Journal of Cognitive Neuroscience' : 'Journal of Cognitive Neuroscience',
        'Cell Death and Disease' : 'Cell Death & Disease',
        'Cell Death and Differentiation' : 'Cell Death & Differentiation',
        'Cell Death Differentiation' : 'Cell Death & Differentiation',
        'Frontiers' : 'Frontiers in Psychology',
        'Journal of cerebral blood flow and metabolism' : 'Journal of Cerebral Blood Flow & Metabolism',
        'Journal of Cerebral Blood Flow and Metabolism' : 'Journal of Cerebral Blood Flow & Metabolism',
        'The Journal of investigative dermatology' : 'Journal of Investigative Dermatology',
        'NATURE COMMUNICATIONS' : 'Nature Communications',
        'Nature Scientific Reports' : 'Scientific Reports',
        'Sci Rep' : 'Scientific Reports',
        'Scientific Reports-11-00861B' : 'Scientific Reports',
        'EMBO reports' : 'EMBO Reports',
        'EMBO Journal' : 'The EMBO Journal',
        'EMBO' : 'The EMBO Journal',
        'The EMBO journal' : 'The EMBO Journal',
        'American Jnl Epidemiology' : 'American Journal of Epidemiology',
        'Brain Online' : 'Brain',
        'Cerebral Cortex print' : 'Cerebral Cortex',
        'Cerebral Cortex Online' : 'Cerebral Cortex',
        'European Heart Journal Online' : 'European Heart Journal',
        'Health Policy Planning' : 'Health Policy and Planning',
        'Oxford Journals' : 'Health Policy and Planning',
        'Human Mol Genetics Online' : 'Human Molecular Genetics',
        'Human molecular genetics' : 'Human Molecular Genetics',
        'Int J Epidemiol' : 'International Journal of Epidemiology',
        'International Joural of Epidemiology': 'International Journal of Epidemiology',
        'Inyernational Journal of Epidemiology' : 'International Journal of Epidemiology',
        'Journal of Infectious Diseases' : 'The Journal of Infectious Diseases',
        'J Infect Dis' : 'The Journal of Infectious Diseases',
        'Nucleic Acid Research' : 'Nucleic Acids Research',
        'NUCLEIC ACIDS RESEARCH' : 'Nucleic Acids Research',
        'Journal of Gerontology Series A: Medical Sciences' : 'The Journals of Gerontology, Series A: Biological Sciences and Medical Sciences',
        'Mol Biol and Evolution' : 'Molecular Biology and Evolution',
        'Nicotine and Tobaco Research' : 'Nicotine & Tobacco Research',
        'Biinformatics' : 'Bioinformatics',
        'Bioinformatics Online' : 'Bioinformatics',
        'Oxford University Press' : 'Age and Ageing',
        'Nephrology Dialysis and Transplantation' : 'Nephrology Dialysis Transplantation',
        'Protein Engineering, Design and Selection (' : 'Protein Engineering, Design and Selection',
        'Environment and Planning A' : 'Environment and Planning A: Economy and Space',
        'PLoS Genetics' : 'PLOS Genetics',
        'Plos Genetics' : 'PLOS Genetics',
        'PLoS GENETICS' : 'PLOS Genetics',
        'PLOS NTD' : 'PLOS Neglected Tropical Diseases',
        'Neglected Tropical Disease' : 'PLOS Neglected Tropical Diseases',
        'PLoS Neglected Tropical Diseases' : 'PLOS Neglected Tropical Diseases',
        'PLOS Neglected Tropical Diseases' : 'PLOS Neglected Tropical Diseases',
        'PNTD' : 'PLOS Neglected Tropical Diseases',
        'PLoS ONE' : 'PLOS One',
        'PloS One' : 'PLOS One',
        'PLoS One' : 'PLOS One',
        'Plos One' : 'PLOS One',
        'PLOS ONE' : 'PLOS One',
        'Plos' : 'PLOS One',
        'PLoS 1' : 'PLOS One',
        'PloS ONE' : 'PLOS One',
        'PLoSONE' : 'PLOS One',
        'PLosONE' : 'PLOS One',
        'PONE-D12-17947' : 'PLOS One',
        'Public Library of Science ONE' : 'PLOS One',
        'PLoS Pathogens' : 'PLOS Pathogens',
        'Plos Pathogens' : 'PLOS Pathogens',
        'BIOCHEM SOC TRANS' : 'Biochemical Society Transactions',
        'Computational Biology' : 'PLOS Computational Biology',
        'PLoS Computational Biology' : 'PLOS Computational Biology',
        'PLOS COMPUTATIONAL BIOLOGY' : 'PLOS Computational Biology',
        'PLoS Biology' : 'PLOS Biology',
        'PLoS Medicine' : 'PLOS Medicine',
        'PLoS Medicine Journal' : 'PLOS Medicine',
        'Pmedicine-D-12-03130' : 'PLOS Medicine',
        'British Journal of Psychiatry' : 'The British Journal of Psychiatry',
        'Britsh Journal of Psychiatry' : 'The British Journal of Psychiatry',
        'BJP' : 'The British Journal of Psychiatry',
        'Interface' : 'Journal of the Royal Society Interface',
        'Journal of the Royal Society, Interface' : 'Journal of the Royal Society Interface',
        'Philos Trans R Soc London B Biol Sci' : 'Philosophical Transactions of the Royal Society B',
        'Philosophical Transactions B' : 'Philosophical Transactions of the Royal Society B',
        'Philosophical Transactions of the Royal Society of London. Series B, Biological Sciences' : 'Philosophical Transactions of the Royal Society B',
        'RSTB 2012.0145' : 'Philosophical Transactions of the Royal Society B',
        'Proceedings B' : 'Proceedings of the Royal Society B',
        'Proceedings of Royal Society' : 'Proceedings of the Royal Society B',
        'Proceedings of the Royal Society': 'Proceedings of the Royal Society B',
        'Proceedings of the Royal Society B: Biological Sciences' : 'Proceedings of the Royal Society B',
        'Chemical Communication' : 'ChemComm',
        'Neuroscientist' : 'The Neuroscientist',
        'American Journal of Tropical Medicine & Hygiene' : 'The American Journal of Tropical Medicine & Hygiene',
        'Am J Trop Med Hyg' : 'The American Journal of Tropical Medicine & Hygiene',
        'Endocrine Related Cancer' : 'Endocrine-Related Cancer',
        'Journal of Microbiology' : 'Microbiology',
        'SGM Open Option' : 'Microbiology',
        'Acta Dermato Venereologica' : 'Acta Dermato-Venereologica',
        'Acta Neuropathol' : 'Acta Neuropathologica',
        'Anal Bioanal Chem.' : 'Analytical and Bioanalytical Chemistry',
        'Cellular and Molecular Cell Sciences' : 'Cellular and Molecular Life Sciences',
        'European Child and Adolescent Psychiatty' : 'European Child and Adolescent Psychiatry',
        'Journal of Autism and Development Disorders' : 'Journal of Autism and Developmental Disorders',
        'Journal of Inherited Metabolic' : 'Journal of Inherited Metabolic Disease',
        'Pflugers Archiv' : 'Pflugers Archive',
        'Social Psychiatry & Psychiatric Epidemiology' : 'Social Psychiatry and Psychiatric Epidemiology',
        'Social Psychiatry and Psychiatric Epidemiol' : 'Social Psychiatry and Psychiatric Epidemiology',
        'Tropical Animal Health & Production' : 'Tropical Animal Health and Production',
        'AIDS Behav' : 'AIDS and Behavior',
        'Springer Science + Business Media' : 'Reproductive Biology and Endocrinology',
        'Childs Nerv Syst' : 'Child\'s Nervous System',
        'AGE' : 'Age',
        'Journal of The American Society for Mass Spectrometry' : 'Journal of the American Society for Mass Spectrometry',
        'Aids Care' : 'AIDS Care',
        'Am J Bioeth' : 'The American Journal of Bioethics',
        'Ethn Health' : 'Ethnicity & Health',
        'American Journal of Bioethics--Neuroscience' : 'AJOB Neuroscience',
        'Dev World Bioeth.' : 'Developing World Bioethics',
        'Dev. World Bioeth' : 'Developing World Bioethics',
        'International Journal of Developmental Biology' : 'The International Journal of Developmental Biology',
        'Acta Physiol' : 'Acta Physiologica',
        'Alimentrary Pharmacology & Therapeutics' : 'Alimentary Pharmacology & Therapeutics',
        'Angewande Chemie' : 'Angewandte Chemie',
        'Arthritis & Rheumatism' : 'Arthritis & Rheumatology',
        'Arthritis and Rheumatism' : 'Arthritis & Rheumatology',
        'Brt Jnl Clinical Pharmacology' : 'British Journal of Clinical Pharmacology',
        'British Jounal of Pharmacology' : 'British Journal of Pharmacology',
        'British journal of Clinical Pharmacology' : 'British Journal of Pharmacology',
        'CHEMBIOCHEM' : 'ChemBioChem',    
        'Chemistry -  European Journal' : 'Chemistry: A European Journal',
        'Child Psychology Psychiatry' : 'Journal of Child Psychology and Psychiatry',
        'The European Journal of Neuroscience' : 'European Journal of Neuroscience',
        'FEBS J.' : 'The FEBS Journal',
        'FEBS Journal' : 'The FEBS Journal',
        'Genes Brain Behaviour' : 'Genes, Brain and Behavior',
        'genesis: Journal of Genetics' : 'Genesis: The Journal of Genetics and Development',
        'HBM JNL Human Brain Mapping' : 'Human Brain Mapping',
        'Human Mutation: Variation, Informatics, and Disease' : 'Human Mutation',
        'JOURNAL OF NEUROCHEMISTRY' : 'Journal of Neurochemistry',
        'Child: Care, Health Development' : 'Child: Care, Health and Development',
        'CHILD: Care, Heath & Development' : 'Child: Care, Health and Development',
        'Maternal and Child Nutrition' : 'Maternal & Child Nutrition',
        'Marten Child Nutr' : 'Maternal & Child Nutrition',
        'Evaluation in Clinical Practice' : 'Journal of Evaluation in Clinical Practice',
        'Journal of Cell Physiology' : 'Journal of Cellular Physiology',
        'Influenza Other Respi Viruses.' : 'Influenza and Other Respiratory Viruses',
        'Clinical Transcriptional Science' : 'Clinical and Translational Science',
        'Acta Opthalmologica' : 'Acta Ophthalmologica',
        'Angew chems Int Ed' : 'Angewandte Chemie International Edition',
        'Criminal Behavior and Mental Health' : 'Criminal Behaviour and Mental Health',
        'Chemmedchem' : 'ChemMedChem', 
        'Protemics' : 'Proteomics',
        'AIDS UK' : 'AIDS',
        'AIDS Journal' : 'JAIDS',
        'JAIDS Journal of Acquired Immune Deficiency Syndromes' : 'JAIDS',
        'Journal of Aids' : 'JAIDS',
        'Journal of Acquired Immune Deficiency Syndroms (JAIDS)' : 'JAIDS',
        'Journal of Acquired Immune Deficiency Syndromes' : 'JAIDS',
        'Pediatr Infect Dis J' : 'The Pediatric Infectious Disease Journal'
               
   
 }, inplace=True)


#***** THIS COMMENT SECTION IS STUFF THAT, IF IT WERE A REAL PROJECT I WOULD ADDRESS. *****
# Elsevier # I find this one hard to believe...
# Brain, Behavior, and Immunity (age related changes...)
# Behaviour Research and Therapy (the role of self-structure)
# The Journal of Pediatrics (Birth status, childgrowth nd)
# Elsevier should be wiley-blackwell (transformation of a plasmid-free)
# Health & Place (are slum dwellers)
# Drug and Alcohol Dependence (clinician and service user perceptions)
# Lancet (Effects of relative weight gain)
# SPRINGER NATURE, not Elsevier # Gene Therapy

### BOOK! ### Macmillan  - Fungal Disease in Britain and the United States 1850-2000    
  
# PLOS but if this is changed first it can mess up #Health below
# Public Library of Science should be PLOS One # The Sec1/Munc18 protein Vps45 regulates cellular levels of its SNARE binding partners Tlg2 and Snc2 in Saccharomyces cerevisiae
# Biology # PLOS but should be PLOS Biology because of MDPI biology
# Understanding support for complementary and alternative medicine in general populations: use and perceived efficacy
# This is from Health, NOT any PLOS publication
# Public Service Review # Laboratory Science in Tropical Medicine #unclear
# Replace Springer with Acta Neuropathologica # Intracerebral immune complex formation induces inflammation in the brain that depends on Fc receptor interaction. 
# AIDS and Behavior is not an article title # Open Choice
# Open Choice #FIX# Detection of antenatal depression in rural HIV affected populations with short and ultra-short versions of the Edingburgh Postnatal Depression Scales (EPDS)

         
##### THESE MAKE NO SENSE
# Science-Research    Increase in adult life expectancy
# Science-Research    High Coverage of ART increases


In [255]:
# Public Health published by BioMed Central is actually BMC Public Health but I won't replace all Public Health
# since that's the actual name of Elsevier's publication. I need to select 'Public Health' in the 'Journal_Title' 
# column where 'Article_Title' is equal to 'Walking and cycling to work despite reporting an unsupportive
# environment: insights from a mixed-method exploration of resilience'.

df.loc[df.Article_Title == 'Walking and cycling to work despite reporting an unsupportive environment: insights from a mixed-method exploration of resilience', 'Journal_Title'] = "BMC Public Health"

In [256]:
# Convert string values in cost column to numeric values after removing $ and GBP signs.
df['Cost_to_Wellcome_in_GBP_plus_VAT'] = df['Cost_to_Wellcome_in_GBP_plus_VAT'].str.replace('£', '')
df['Cost_to_Wellcome_in_GBP_plus_VAT'] = df['Cost_to_Wellcome_in_GBP_plus_VAT'].str.replace('$', '')
df['Cost_to_Wellcome_in_GBP_plus_VAT'] = pd.to_numeric(df['Cost_to_Wellcome_in_GBP_plus_VAT'])

# Some cost amounts are 999999.00 and some are 0 - exclude these. Replace with NaN and drop.
df['Cost_to_Wellcome_in_GBP_plus_VAT'] = df['Cost_to_Wellcome_in_GBP_plus_VAT'].replace(999999.00, np.nan)
df['Cost_to_Wellcome_in_GBP_plus_VAT'] = df['Cost_to_Wellcome_in_GBP_plus_VAT'].replace(0, np.nan)

df = df.dropna()
df.head(2)

,Publisher,Journal_Title,Article_Title,Cost_to_Wellcome_in_GBP_plus_VAT
1,ACS,Biomacromolecules,Structural characterization of a Model Gram-ne...,2381.04
2,ACS,Journal of Medicinal Chemistry,"Fumaroylamino-4,5-epoxymorphinans and related ...",642.56


###  #1. Using the dataset of article open-access prices paid by the WELLCOME Trust between 2012 and 2013, determine the five most common journals and the total articles for each. 

In [257]:
# Count all articles by journal, sort descending, display top 5 journals based on highest article counts. 
# PLOS One has 197 articles, JBC 71, PNAS 39, NeuroImage 31, and Nucleic Acids Research 29. 

journal_count = df.groupby(df['Journal_Title']).count()
journal_descend = journal_count.sort_values(by='Article_Title', ascending=False)
journal_descend.head()

,Publisher,Article_Title,Cost_to_Wellcome_in_GBP_plus_VAT
Journal_Title,,,
PLOS One,197,197,197
Journal of Biological Chemistry,71,71,71
Proceedings of the National Academy of Sciences of the United States of America,39,39,39
NeuroImage,31,31,31
Nucleic Acids Research,29,29,29


### #2. Next, calculate the mean, median, and standard deviation of the open-access cost per article for each journal. 

In [258]:
# Just out of curiosity, what are the 10 most expensive articles?
df.sort_values(by='Cost_to_Wellcome_in_GBP_plus_VAT', ascending=False).head(10)

,Publisher,Journal_Title,Article_Title,Cost_to_Wellcome_in_GBP_plus_VAT
1987,Wiley,Movement Disorders,Limb amputations in fixed dystonia: a form of ...,201024.00
1470,PLOS,PLOS One,Reducing stock-outs of life saving Malaria Com...,192645.00
1619,PSCA International,Public Service Review,Laboratory Science in Tropical Medicine,6000.00
800,Elsevier,The Lancet Neurology,Genetic risk factors for ischaemic stroke and ...,5760.00
798,Elsevier,The Lancet,Effects of unconditional and conditional cash ...,4800.00
648,Elsevier,The Lancet,Global trends in antiretroviral resistance in ...,4800.00
552,Elsevier,Elsevier,Effects of relative weight gain and linear gro...,4800.00
797,Elsevier,The Lancet,The impact on a serogroup a meningococcal conj...,4554.01
796,Elsevier,The Lancet,Dementia incidence and mortality in middle inc...,4320.00
799,Elsevier,The Lancet Neurology,MRI investigation of the sensorimotor cortex a...,4320.00


In [259]:
result = df.groupby('Journal_Title', as_index=False).agg([np.count_nonzero,
                                                       np.mean, 
                                                       np.median, 
                                                       lambda x: np.std(x, ddof=0)])


In [260]:
# COST PER ARTICLE BY JOURNAL RESULTS:
# NeuroImage: mean - 2212.181290, median - 2326.43, standard deviation - 268.750782
# PLOS One: mean - 1904.623096, median - 896.99, standard deviation - 13625.597984
# JBC: mean - 1378.140423, median - 1311.73, standard deviation - 385.807913
# Nucleic Acids Research: mean - 1162.344828, median - 852.00, standard deviation - 434.460766
# PNAS: mean - 839.883333, median - 734.00, standard deviation - 449.444482

# Print final report in descending order by median cost for each journal:
# The column titled "<lambda>" is Standard Deviation.
final_report = result['Cost_to_Wellcome_in_GBP_plus_VAT'].sort_values(['count_nonzero'], ascending = False)

final_report[:5]

,count_nonzero,mean,median,<lambda>
Journal_Title,,,,
PLOS One,197.0,1904.623096,896.99,13625.597984
Journal of Biological Chemistry,71.0,1378.140423,1311.73,385.807913
Proceedings of the National Academy of Sciences of the United States of America,39.0,839.883333,734.00,449.444482
NeuroImage,31.0,2212.181290,2326.43,268.750782
Nucleic Acids Research,29.0,1162.344828,852.00,434.460766


### Open Access cost per article by journal looked at another way...

In [261]:
# VALUES FOR PLOS One: 
print("mean of Open Access article cost: ", df[df['Journal_Title'] =='PLOS One'].mean())
print("median of Open Access article cost: ", df[df['Journal_Title'] =='PLOS One'].median())
print("standard deviation of Open Access article cost: ", df[df['Journal_Title'] =='PLOS One'].std(ddof=0))


mean of Open Access article cost:  Cost_to_Wellcome_in_GBP_plus_VAT    1904.623096
dtype: float64
median of Open Access article cost:  Cost_to_Wellcome_in_GBP_plus_VAT    896.99
dtype: float64
standard deviation of Open Access article cost:  Cost_to_Wellcome_in_GBP_plus_VAT    13625.597984
dtype: float64


In [262]:
# VALUES FOR Journal of Biological Chemistry: 
print("mean: ", df[df['Journal_Title'] =='Journal of Biological Chemistry'].mean())
print("median: ", df[df['Journal_Title'] =='Journal of Biological Chemistry'].median())
print("standard deviation: ", df[df['Journal_Title'] =='Journal of Biological Chemistry'].std(ddof=0))


mean:  Cost_to_Wellcome_in_GBP_plus_VAT    1378.140423
dtype: float64
median:  Cost_to_Wellcome_in_GBP_plus_VAT    1311.73
dtype: float64
standard deviation:  Cost_to_Wellcome_in_GBP_plus_VAT    385.807913
dtype: float64


In [263]:
# VALUES FOR Proceedings of the National Academy of Sciences of the United States of America: 
print("mean of Open Access article cost: ", df[df['Journal_Title'] =='Proceedings of the National Academy of Sciences of the United States of America'].mean())
print("median of Open Access article cost: ", df[df['Journal_Title'] =='Proceedings of the National Academy of Sciences of the United States of America'].median())
print("standard deviation of Open Access article cost: ", df[df['Journal_Title'] =='Proceedings of the National Academy of Sciences of the United States of America'].std(ddof=0))

mean of Open Access article cost:  Cost_to_Wellcome_in_GBP_plus_VAT    839.883333
dtype: float64
median of Open Access article cost:  Cost_to_Wellcome_in_GBP_plus_VAT    734.0
dtype: float64
standard deviation of Open Access article cost:  Cost_to_Wellcome_in_GBP_plus_VAT    449.444482
dtype: float64


In [264]:
# VALUES FOR NeuroImage: 
print("mean of Open Access article cost: ", df[df['Journal_Title'] =='NeuroImage'].mean())
print("median of Open Access article cost: ", df[df['Journal_Title'] =='NeuroImage'].median())
print("standard deviation of Open Access article cost: ", df[df['Journal_Title'] =='NeuroImage'].std(ddof=0))

mean of Open Access article cost:  Cost_to_Wellcome_in_GBP_plus_VAT    2212.18129
dtype: float64
median of Open Access article cost:  Cost_to_Wellcome_in_GBP_plus_VAT    2326.43
dtype: float64
standard deviation of Open Access article cost:  Cost_to_Wellcome_in_GBP_plus_VAT    268.750782
dtype: float64


In [265]:
# VALUES FOR Nucleic Acids Research: 
print("mean of Open Access article cost: ", df[df['Journal_Title'] =='Nucleic Acids Research'].mean())
print("median of Open Access article cost: ", df[df['Journal_Title'] =='Nucleic Acids Research'].median())
print("standard deviation of Open Access article cost: ", df[df['Journal_Title'] =='Nucleic Acids Research'].std(ddof=0))

mean of Open Access article cost:  Cost_to_Wellcome_in_GBP_plus_VAT    1162.344828
dtype: float64
median of Open Access article cost:  Cost_to_Wellcome_in_GBP_plus_VAT    852.0
dtype: float64
standard deviation of Open Access article cost:  Cost_to_Wellcome_in_GBP_plus_VAT    434.460766
dtype: float64
